# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)


## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)



606

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [3]:
##PERFORM API CALLS
#Define URL
url = "http://api.openweathermap.org/data/2.5/weather?"
city = "timizart"

api_url = url + "appid=" + weather_api_key + "&q=" + city + "&units=imperial"

#Perform API call with json response
weather_response = requests.get(api_url)
weather_json = weather_response.json()
weather_json

{'coord': {'lon': 4.27, 'lat': 36.8},
 'weather': [{'id': 803,
   'main': 'Clouds',
   'description': 'broken clouds',
   'icon': '04n'}],
 'base': 'model',
 'main': {'temp': 57.72,
  'feels_like': 52.5,
  'temp_min': 57.72,
  'temp_max': 57.72,
  'pressure': 1028,
  'humidity': 58,
  'sea_level': 1028,
  'grnd_level': 1028},
 'wind': {'speed': 6.42, 'deg': 102},
 'clouds': {'all': 73},
 'dt': 1579199287,
 'sys': {'country': 'DZ', 'sunrise': 1579157670, 'sunset': 1579193398},
 'timezone': 3600,
 'id': 2476396,
 'name': 'Timizart',
 'cod': 200}

In [4]:
##CONVERT RAW DATA TO A DATAFRAME
#Create df with cities data and NaN data for rest of the columns
cities_df = pd.DataFrame({
    "City":cities,
    "Temp":np.nan,
    "Humidity":np.nan,
    "Cloudiness":np.nan,
    "Wind Speed":np.nan
})


In [5]:
#WRITE API CALL DATA TO DF
#Start for loop for number of row in previous df
for index, row in cities_df[0:631].iterrows():
    try:
        #All query and param data must be within indentation
        row = row.copy()
        city= row['City']
        print(city)

        url = "http://api.openweathermap.org/data/2.5/weather?"

        query_url = url + "appid=" + weather_api_key + "&q=" + city + "&units=imperial"
        print(query_url)
        
        #API call
        weather_response = requests.get(query_url)
        weather_json = weather_response.json()
        weather_json

        #Write info to df
        cities_df.at[index,"Temp"]= weather_json["main"]["temp"]
        cities_df.at[index,"Humidity"]= weather_json["main"]["humidity"]
        cities_df.at[index,"Cloudiness"]= weather_json["clouds"]["all"]
        cities_df.at[index,"Wind Speed"]= weather_json["wind"]["speed"]
        cities_df.at[index,"lat"]= weather_json["coord"]["lat"]
        cities_df.at[index,"lon"]= weather_json["coord"]["lon"]
        
    #Error exception for cities with no data
    except KeyError:
        print(f"No data in, {row.copy()} keep NaN in df.")

#Display the df
cities_df

port alfred
http://api.openweathermap.org/data/2.5/weather?appid=72930d4a88a7446428ea57e3fa6f594f&q=port alfred&units=imperial
oranjemund
http://api.openweathermap.org/data/2.5/weather?appid=72930d4a88a7446428ea57e3fa6f594f&q=oranjemund&units=imperial
russell
http://api.openweathermap.org/data/2.5/weather?appid=72930d4a88a7446428ea57e3fa6f594f&q=russell&units=imperial
yellowknife
http://api.openweathermap.org/data/2.5/weather?appid=72930d4a88a7446428ea57e3fa6f594f&q=yellowknife&units=imperial
ushuaia
http://api.openweathermap.org/data/2.5/weather?appid=72930d4a88a7446428ea57e3fa6f594f&q=ushuaia&units=imperial
longyearbyen
http://api.openweathermap.org/data/2.5/weather?appid=72930d4a88a7446428ea57e3fa6f594f&q=longyearbyen&units=imperial
benjamin hill
http://api.openweathermap.org/data/2.5/weather?appid=72930d4a88a7446428ea57e3fa6f594f&q=benjamin hill&units=imperial
bengkulu
http://api.openweathermap.org/data/2.5/weather?appid=72930d4a88a7446428ea57e3fa6f594f&q=bengkulu&units=imperial
ma

,City,Temp,Humidity,Cloudiness,Wind Speed,lat,lon
0,port alfred,73.99,92.0,0.0,5.99,-33.59,26.89
1,oranjemund,66.49,92.0,34.0,15.10,-28.55,16.43
2,russell,69.73,60.0,1.0,12.75,32.35,-85.20
3,yellowknife,-38.20,65.0,75.0,4.70,62.46,-114.35
4,ushuaia,49.19,71.0,90.0,4.07,-54.80,-68.30
5,longyearbyen,-2.20,77.0,15.0,8.05,78.22,15.64
6,benjamin hill,62.01,51.0,100.0,8.99,30.17,-111.10
7,bengkulu,73.45,94.0,94.0,3.00,-3.80,102.27
8,mata grande,98.60,25.0,40.0,8.05,-9.12,-37.73
9,new norfolk,45.90,71.0,40.0,9.17,-42.78,147.06


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [23]:
#Export weather_data to csv_file and remove index
cities_df.to_csv("WeatherPy.csv", sep=',')


In [27]:
cities_df.head()
cities_df.to_csv('../output_data/cities.csv')

### Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

#### Latitude vs. Temperature Plot

In [28]:
##LATITUDE VS. TEMPERATURE PLOT
plt.scatter(cities_df['lat'], cities_df['Temp'], marker="o", facecolors="r", edgecolors="black")

#Plot
plt.title("Temperature (F) vs. Latitude Date of Analysis 24Feb2019")
plt.xlabel("Latitude")
plt.ylabel("Temperature (F)")
plt.savefig("../output_data/LatitudeVCloudiness.png")
plt.close()



#### Latitude vs. Humidity Plot

In [29]:
##LATITUDE VS. HUMIDITY PLOT
plt.scatter(cities_df['lat'], cities_df['Humidity'], marker="o", facecolors="g", edgecolors="black")

#Plot
plt.title("Humidity % vs. Latitude Date of Analysis 24Feb2019")
plt.xlabel("Latitude")
plt.ylabel("Humidity %")
plt.savefig("../output_data/LatitudeVHumidity.png")
plt.close()


#### Latitude vs. Cloudiness Plot

In [30]:
##LATITUDE VS. CLOUDINESS PLOT
plt.scatter(cities_df['lat'], cities_df['Cloudiness'], marker="o", facecolors="b", edgecolors="black")

#Format scatter plot
plt.title("Cloudiness % vs. Latitude Date of Analysis 24Feb2019")
plt.xlabel("Latitude")
plt.ylabel("Cloudiness %")
plt.savefig("../output_data/LatitudeVCloud.png")
plt.close()

#### Latitude vs. Wind Speed Plot

In [31]:
##LATITUDE VS. WIND SPEED PLOT
plt.scatter(cities_df['lat'], cities_df['Wind Speed'], marker="o", facecolors="orange", edgecolors="black")

#Format scatter plot
plt.title("Wind Speed vs. Latitude Date of Analysis 24Feb2019")
plt.xlabel("Latitude")
plt.ylabel("Wind Speed (mph)")
plt.savefig("../output_data/LatitudeVWind.png")
plt.close()